# USE A NORMALIZED VERSION OF THE DATA AND EPOCH = 100

In [1]:
# import library
import pandas as pd
import numpy as np

In [2]:
concrete_data = pd.read_csv('https://cocl.us/concrete_data')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [3]:
concrete_data.shape

(1030, 9)

In [4]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [5]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

In [6]:
concrete_data_columns = concrete_data.columns
predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

In [7]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [8]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

In [9]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [10]:
# number of predictors
n_cols = predictors_norm.shape[1]

In [11]:
import keras

Using TensorFlow backend.
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyterlab/con

In [12]:
from keras.models import Sequential
from keras.layers import Dense

In [13]:
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=42)

In [16]:
model = regression_model()

In [17]:
epochs = 50
model.fit(X_train, y_train, epochs=epochs, verbose=2)

Epoch 1/50
 - 1s - loss: 1589.3960
Epoch 2/50
 - 0s - loss: 1574.1271
Epoch 3/50
 - 0s - loss: 1559.6761
Epoch 4/50
 - 0s - loss: 1545.3454
Epoch 5/50
 - 0s - loss: 1531.3782
Epoch 6/50
 - 0s - loss: 1517.4473
Epoch 7/50
 - 0s - loss: 1503.1597
Epoch 8/50
 - 0s - loss: 1488.4720
Epoch 9/50
 - 0s - loss: 1473.0822
Epoch 10/50
 - 0s - loss: 1457.3502
Epoch 11/50
 - 0s - loss: 1440.4938
Epoch 12/50
 - 0s - loss: 1422.9547
Epoch 13/50
 - 0s - loss: 1404.0401
Epoch 14/50
 - 0s - loss: 1384.0724
Epoch 15/50
 - 0s - loss: 1363.3393
Epoch 16/50
 - 0s - loss: 1340.6765
Epoch 17/50
 - 0s - loss: 1317.9199
Epoch 18/50
 - 0s - loss: 1293.7126
Epoch 19/50
 - 0s - loss: 1268.7498
Epoch 20/50
 - 0s - loss: 1242.2700
Epoch 21/50
 - 0s - loss: 1215.3890
Epoch 22/50
 - 0s - loss: 1187.2256
Epoch 23/50
 - 0s - loss: 1158.4201
Epoch 24/50
 - 0s - loss: 1129.0969
Epoch 25/50
 - 0s - loss: 1098.8302
Epoch 26/50
 - 0s - loss: 1067.9506
Epoch 27/50
 - 0s - loss: 1036.4282
Epoch 28/50
 - 0s - loss: 1004.6293
E

In [18]:
loss_val = model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
loss_val

309/309 [==============================] - 0s 226us/step


316.8251930409651

In [19]:
from sklearn.metrics import mean_squared_error
mean_square_error = mean_squared_error(y_test, y_pred)
mean = np.mean(mean_square_error)
standard_deviation = np.std(mean_square_error)
print(mean, standard_deviation)

316.82519510358804 0.0


In [20]:

total_mean_squared_errors = 50
epochs = 100
mean_squared_errors = []
for i in range(0, total_mean_squared_errors):
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=i)
    model.fit(X_train, y_train, epochs=epochs, verbose=0)
    MSE = model.evaluate(X_test, y_test, verbose=0)
    print("MSE "+str(i+1)+": "+str(MSE))
    y_pred = model.predict(X_test)
    mean_square_error = mean_squared_error(y_test, y_pred)
    mean_squared_errors.append(mean_square_error)

mean_squared_errors = np.array(mean_squared_errors)
mean = np.mean(mean_squared_errors)
standard_deviation = np.std(mean_squared_errors)

print('\n')
print("Below is the mean and standard deviation of " +str(total_mean_squared_errors) + " mean squared errors with normalized data. Total number of epochs for each training is: " +str(epochs) + "\n")
print("Mean: "+str(mean))
print("Standard Deviation: "+str(standard_deviation))

MSE 1: 102.63063207336228
MSE 2: 89.09010396420376
MSE 3: 70.13795960065231
MSE 4: 73.962036194539
MSE 5: 70.9901605254238
MSE 6: 58.325711839793186
MSE 7: 53.9393995957853
MSE 8: 35.282438839909325
MSE 9: 39.05247335835182
MSE 10: 38.487691737301525
MSE 11: 42.38710818244416
MSE 12: 35.825191868161696
MSE 13: 43.51650433092827
MSE 14: 44.51197351992709
MSE 15: 38.07049043278864
MSE 16: 32.854976567635646
MSE 17: 36.221844682415714
MSE 18: 35.53472832491483
MSE 19: 36.506166637136715
MSE 20: 39.53768080183603
MSE 21: 33.41991878867535
MSE 22: 35.05298400928287
MSE 23: 31.804095765147782
MSE 24: 37.0727442769171
MSE 25: 34.61937214255719
MSE 26: 38.18197091111859
MSE 27: 32.65790755386106
MSE 28: 33.67852485681429
MSE 29: 39.62035101980067
MSE 30: 38.698031447080346
MSE 31: 35.30298878074078
MSE 32: 33.523983890570484
MSE 33: 33.962910809563205
MSE 34: 35.309886833610655
MSE 35: 37.21025739750044
MSE 36: 41.62757714280804
MSE 37: 32.520081171325884
MSE 38: 37.65882532804915
MSE 39: 34.8